In [ ]:
pip install pymupdf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.1/24.1 MB 42.2 MB/s eta 0:00:00


In [ ]:
import fitz  # PyMuPDF

def extract_text_from_pdf(pdf_path):
    doc = fitz.open(pdf_path)  # <--- This line uses the path passed as an argument
    full_text = ""
    for page in doc:
        full_text += page.get_text()
    return full_text

In [ ]:
import re

def chunk_text(text, max_words=100):
    words = text.split()
    return [' '.join(words[i:i+max_words]) for i in range(0, len(words), max_words)]

In [ ]:
pip install sentence-transformers faiss-cpu


In [ ]:
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np

def embed_chunks(chunks, model):
    embeddings = model.encode(chunks, show_progress_bar=True)
    embeddings = np.array(embeddings).astype("float32")
    return embeddings

def create_faiss_index(embeddings):
    index = faiss.IndexFlatL2(embeddings.shape[1])
    index.add(embeddings)
    return index


In [ ]:
def search_faiss(query, model, index, chunks, k=5):
    query_vector = model.encode([query]).astype("float32")
    D, I = index.search(query_vector, k)
    results = [chunks[i] for i in I[0]]
    return results


In [ ]:
pip install groq


In [ ]:
def ask_llm(chunks, model, index):
    question = "What is metagenomics and provide it's use cases in 3 bulletin points"
    query_vec = model.encode([question]).astype("float32")
    D, I = index.search(query_vec, 5)
    context = "\n".join([chunks[i] for i in I[0]])

    prompt = f"""Answer the following question using the provided document context.

Context:
{context}

Question: {question}

Answer:"""

    response = client.chat.completions.create(
        model="llama3-70b-8192",
        messages=[{"role": "user", "content": prompt}]
    )

    return response.choices[0].message.content.strip()

# ----------------------------------------------
# 7. PIPELINE EXECUTION — YOUR PDF PATH HERE
# ----------------------------------------------
pdf_path = "/content/bbad351.pdf"

# Extract text
text = extract_text_from_pdf(pdf_path)

# Chunk text
chunks = chunk_text(text)

# Load embedding model
model = SentenceTransformer("all-MiniLM-L6-v2")

# Embed & index
embeddings = embed_chunks(chunks, model)
index = create_faiss_index(embeddings)

# Ask question
answer = ask_llm(chunks, model, index)
print("🧠 Answer:", answer)

Batches:   0%|          | 0/3 [00:00<?, ?it/s]

🧠 Answer: Metagenomics is not explicitly defined in the provided document context, but based on the content, it can be inferred that metagenomics is the study of microbial communities through the analysis of genetic material recovered directly from environmental samples.

Here are three use cases of metagenomics in bulletin points:

• **Understanding microbe-microbe interactions**: Metagenomics can help understand the interactions between microbes and their roles in physiological, metabolic, ecological, and evolutionary contexts.

• **Elucidating host-microbe-microbe co-evolution**: Metagenomics can shed light on the co-evolution of hosts and microbes, providing insights into the complex relationships between them.

• **Analyzing environmental samples**: Metagenomics can be used to analyze environmental samples, such as soil, water, or air, to understand the composition and diversity of microbial communities in these environments.
